<a href="https://colab.research.google.com/github/ldmrepo/java-tutorial/blob/main/vec_dist_weight_apply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. **가중치 행렬 \( \mathbf{W} \)**:
   이 행렬은 각 요소에 적용할 가중치를 담고 있습니다. 요소 간 거리에 따라 가중치가 결정됩니다. 예를 들어, 5차원 벡터에 대한 가중치 행렬은 다음과 같이 표현될 수 있습니다:
   $$
   \mathbf{W} = \begin{bmatrix}
   0 & w_{1,2} & w_{1,3} & w_{1,4} & w_{1,5} \\
   w_{2,1} & 0 & w_{2,3} & w_{2,4} & w_{2,5} \\
   w_{3,1} & w_{3,2} & 0 & w_{3,4} & w_{3,5} \\
   w_{4,1} & w_{4,2} & w_{4,3} & 0 & w_{4,5} \\
   w_{5,1} & w_{5,2} & w_{5,3} & w_{5,4} & 0 \\
   \end{bmatrix}
   $$
   여기서 \( w_{i,j} \)는 i번째 요소와 j번째 요소 사이의 거리에 반비례하는 가중치입니다.

2. **가중치 적용**:
   벡터 \( \mathbf{v} \)에 가중치 행렬 \( \mathbf{W} \)를 적용해 새로운 벡터 \( \mathbf{v}' \)를 계산합니다. 이는 행렬 곱셈으로 표현됩니다:
   $$
   \mathbf{v}' = \mathbf{W} \mathbf{v}
   $$

3. **원래 값 유지**:
   벡터 \( \mathbf{v} \)의 원래 값이 있는 위치를 \( \mathbf{v}' \)에서 유지합니다. 만약 \( \mathbf{v} \)의 최대값이 있는 인덱스가 \( i \)라면, \( \mathbf{v}'[i] = \mathbf{v}[i] \)로 설정합니다.

4. **무작위성 추가**:
   각 요소에 무작위 값을 더하여 새로운 벡터 \( \mathbf{v}'' \)를 생성합니다:
   $$
   \mathbf{v}'' = \mathbf{v}' + \text{Random}(-\epsilon, \epsilon)
   $$
   여기서 \( \epsilon \)은 작은 상수이며, \(\text{Random}(-\epsilon, \epsilon)\)는 각 요소에 무작위로 선택된 값을 의미합니다.

5. **정규화**:
   \( \mathbf{v}'' \)의 모든 요소의 합이 1이 되도록 정규화합니다:
   $$
   \mathbf{v}'' = \frac{\mathbf{v}''}{\sum \mathbf{v}''}
   $$


In [6]:
import numpy as np

"""
1. 거리 가중치 행렬 생성
2. 입력된 벡터들에 대해 가중치 행렬을 적용
3. 각 벡터의 원래 값이 있는 위치를 유지
4. 무작위성 추가
"""
def create_weight_matrix(size):
    """ 주어진 크기에 대한 가중치 행렬 생성 """
    return np.array([[0 if i == j else 1 / (abs(i - j) + 1) for j in range(size)] for i in range(size)])

def apply_weights(v, weight_matrix):
    """ 벡터에 가중치 행렬 적용 """
    return np.dot(weight_matrix, v)

def preserve_original_value(v, weighted_v):
    """ 원래 값이 있는 위치의 요소 유지 """
    original_value_index = np.argmax(v)
    original_value = v[original_value_index]
    weighted_v -= weighted_v[original_value_index]  # 원래 값 제거
    weighted_v /= np.sum(weighted_v)  # 정규화
    weighted_v *= (1 - original_value)  # 원래 값 고려하여 조정
    weighted_v[original_value_index] = original_value  # 원래 값 복원
    return weighted_v

def add_randomness(v, randomness_scale=0.01):
    """ 벡터에 무작위성 추가 """
    random_v = v + np.random.uniform(-randomness_scale, randomness_scale, v.shape)
    random_v[random_v < 0] = 0  # 음수 방지
    random_v /= np.sum(random_v)  # 정규화
    return random_v

def process_vectors(vectors, weight_matrix, randomness_scale=0.01):
    """ 벡터들을 처리하는 메인 함수 """
    return [add_randomness(preserve_original_value(v, apply_weights(v, weight_matrix)), randomness_scale) for v in vectors]

# 벡터와 가중치 행렬
vectors = np.array([
    [0, 0.6, 0, 0, 0],
    [0, 0, 0, 0.7, 0],
    [0.9, 0, 0, 0, 0]
])
weight_matrix = create_weight_matrix(5)

# 처리된 벡터들
processed_vectors = process_vectors(vectors, weight_matrix)
processed_vectors

[[0.         0.5        0.33333333 0.25       0.2       ]
 [0.5        0.         0.5        0.33333333 0.25      ]
 [0.33333333 0.5        0.         0.5        0.33333333]
 [0.25       0.33333333 0.5        0.         0.5       ]
 [0.2        0.25       0.33333333 0.5        0.        ]]
